<a href="https://colab.research.google.com/github/andresmg42/Proyectos-ADAII/blob/main/final_bruteforce_algorithm_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECT 1(DINAMIC AND GREEDY PROGRAMING)

In [3]:
# structure data definition

#inputs:

#M: list of tuples (Mi,mi)

#E: list of tuples of (ej,msj)

class Subject:
  def __init__(self,code,quotas):
    self.code:str=code
    self.quotas:int=quotas

class Student:
  def __init__(self,code,solicited_subjects,insatisfaction=0):
    self.code:str=code
    self.solicited_subjects:List[SolicitedSubject]=solicited_subjects
    self.assigned_subjects:List[SolicitedSubject]=[]


  def return_info(self):
    string=f'\n{self.code}\n'
    for s_subject in self.solicited_subjects:
       string+=s_subject.return_info()
    return string

  def return_info_sol(self):
    string=f'\n{self.code}\n'
    for s_subject in self.assigned_subjects:
       string+=s_subject.return_info()
    return string


  def calculate_insatisfaction(self):

    if len(self.solicited_subjects)==0:
      return 0.0

    assigned_subject_codes={subject.code for subject in self.assigned_subjects}
    maj_size=len(self.assigned_subjects)
    msj_size=len(self.solicited_subjects)
    if msj_size==0:
      return 0.0
    unsupplied_subjects=[solicited for solicited in self.solicited_subjects if solicited.code not in assigned_subject_codes ]
    sum_pjl=sum(solicited_subject.priority for solicited_subject in unsupplied_subjects)
    fun_Y= msj_size*3-1

    return (1-(maj_size/msj_size))*(sum_pjl/fun_Y)


  def assign_subject(self,subject):

    solicited_subject=next((solicited_subject for solicited_subject in self.solicited_subjects if solicited_subject.code==subject.code),None)
    if solicited_subject:
      self.assigned_subjects.append(solicited_subject)


class SolicitedSubject:
  def __init__(self,subject,priority):
    self.subject=subject
    self.priority:int=priority


  def return_info(self):
    return f'({self.subject.code},{self.priority})'

  @property
  def code(self) -> str:
    return self.subject.code

In [4]:
from typing import List
#implement an example to test
subjects=[
  Subject('M1',3),
  Subject('M2',4),
  Subject('M3',2)
]

students=[
    Student('e1',[SolicitedSubject(subjects[0],5),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],1)
                  ],1),
    Student('e2',[SolicitedSubject(subjects[0],4),
                  SolicitedSubject(subjects[1],1),
                  SolicitedSubject(subjects[2],3)
                  ],2),
    Student('e3',[SolicitedSubject(subjects[1],3),
                  SolicitedSubject(subjects[2],2)
                  ],3),
    Student('e4',[SolicitedSubject(subjects[0],2),
                  SolicitedSubject(subjects[2],3)
                  ],4),
    Student('e5',[SolicitedSubject(subjects[0],3),
                  SolicitedSubject(subjects[1],2),
                  SolicitedSubject(subjects[2],3)
                  ],5)

]

In [11]:
from re import sub
import copy
import math

class brute_force_algorithm:

  def __init__(self) -> None:
    pass


  def combinations(self,elements, k):

      if k == 0:
          return [[]]

      if len(elements) < k:
          return []

      first = elements[0]
      remain = elements[1:]

      whit_first = [[first] + c for c in self.combinations(remain, k-1)]

      whitout_first = self.combinations(remain, k)

      return whit_first + whitout_first


  def combinations_by_subject(self,subject,students):
    quotes=subject.quotas
    code=subject.code
    applicant_students=[student for student in students if any([solicitedsubject.code==code for solicitedsubject in student.solicited_subjects])]

    comb =self.combinations([student.code for student in applicant_students],quotes)

    return comb

  def distribution_final(self,distributions):

    if len(distributions)==0:
      return [[]]

    first_array=distributions[0]

    rest_array=distributions[1:]

    rest_combines=self.distribution_final(rest_array)

    result=[]
    for element in first_array:
      for comb in rest_combines:
        result.append([element]+comb)

    return result


  def distribute_solutions(self,subjects,students):
    subject_distribution=[]

    for subject in subjects:
      posible_comb=self.combinations_by_subject(subject,students)
      subject_distribution.append(posible_comb)


    return subject_distribution


  def get_final_solutions(self,distributed_solutions):
    final_solutions=[]
    for solution in distributed_solutions:
      new_students=copy.deepcopy(students)
      for i in range(len(solution)):
        for student in new_students:
          subject=subjects[i]
          if student.code in solution[i]:
            student.assign_subject(subject)
      final_solutions.append(new_students)
    return final_solutions

  def find_optimal_solution(self,final_solutions,students):
    min_general_satisfaction={'min':math.inf,'solution':None}
    for solution in final_solutions:
      acumulated_insatisfaction=0
      for i in range(len(solution)):

        student=solution[i]

        student_insatisfaction=student.calculate_insatisfaction()

        acumulated_insatisfaction+=student_insatisfaction

      general_insatisfaction=acumulated_insatisfaction/len(students)
      if min_general_satisfaction['min']>general_insatisfaction:
        min_general_satisfaction['min']=general_insatisfaction
        min_general_satisfaction['solution']=solution

    return min_general_satisfaction


  def bruteforce_pipeline(self,subjects,students):

    subject_distribution= self.distribute_solutions(subjects,students)

    distributed_solutions=self.distribution_final(subject_distribution)

    final_solutions=self.get_final_solutions(distributed_solutions)

    optimal_solution=self.find_optimal_solution(final_solutions,students)

    return optimal_solution




In [12]:
brute_force=brute_force_algorithm()

sol=brute_force.bruteforce_pipeline(subjects,students)

print(sol['min'])
for student in sol['solution']:
  print(student.return_info_sol())

0.09833333333333334

e1
(M1,5)(M2,2)

e2
(M1,4)(M2,1)

e3
(M2,3)

e4
(M1,2)(M3,3)

e5
(M2,2)(M3,3)
